<a href="https://colab.research.google.com/github/OttmarIvey/MNIST-con-TensorFLow/blob/main/MNIST_con_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Cargamos el dataset de MNIST

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


# Preprocesamiento (permutacion y creacion de lotes)

In [100]:
mnist_entrenamiento = mnist_dataset['train']
mnist_prueba =  mnist_dataset['test']

num_muestras_validacion = 0.1*mnist_info.splits['train'].num_examples #Extrae el 10% del dataset de prueba
num_muestras_validacion = tf.cast(num_muestras_validacion,tf.int64) #Hacemos que el numero se4a un enetero

num_muestras_prueba = mnist_info.splits['test'].num_examples #Guardamos el numero de muestras de prueba en una variable
num_muestras_prueba = tf.cast(num_muestras_prueba,tf.int64) #Nos aseguramos de que sea un numero entero

#Ahora creamos una funcion que escale las entradas entre 0 y 1

def escala(image, label):
  image = tf.cast(image,tf.float32) #Para que todos los numeros sean reales
  image /=255. #Dividimos cada elemento de la imagen entre 255 ya que es el maximo valor que puede tener.
  return image,label

datos_escalados_entrenamiento_validacion = mnist_entrenamiento.map(escala)
datos_prueba = mnist_prueba.map(escala)

#Ahora vamos a permutar los datos del dataset MNIST aleatoriamente ya que es posible que este organizado de alguna forma.
#lo cual confundira al modelo.

BUFFER_SIZE = 10000

datos_entrenamiento_validacion_permutados = datos_escalados_entrenamiento_validacion.shuffle(BUFFER_SIZE)

datos_validacion = datos_entrenamiento_validacion_permutados.take(num_muestras_validacion) #Extraemos los datos de validacion

datos_entrenamiento = datos_entrenamiento_validacion_permutados.skip(num_muestras_validacion) #Extraemos los datos de entrenamiento saltando los primeros datos de validacion


#Crearemos los distintos lotes de entrenamiento, validacion y prueba
# Un tamaño de lote igual a 1 corresponde al Descenso del gradiente Estocastico
# Un tamaño igual al numero de muestras corresponde al Descenso del Gradiente
# Un tamñao mayor a 1 y menor al numero de muestras corresponde al Descenso del Gradiente por mini lotes
tamaño_lote = 100

datos_entrenamiento = datos_entrenamiento.batch(tamaño_lote) #añade una nueva columna a nuestro tensor para que el alogritmo sepa cuantos datos tomar en cada lote
datos_validacion = datos_validacion.batch(num_muestras_validacion) #Añade otra columna al tensor indicando que debe tomar todos los datos de validacion a la vez
datos_prueba = datos_prueba.batch(num_muestras_prueba)


#Ahora extraemos y convertimos las entradas y objetivos en la forma adecuada
entradas_validacion, objetivos_validacion = next(iter(datos_validacion)) #next carga el siguiente elemento de un objetivo iterable


# Modelo

In [101]:
#Tamaño de los hiperparametros
tamaño_entrada = 784 #Hay 784 entradas en nuestra red
tamaño_salida = 10 # 1 salida por cada digito
tamaño_capa_oculta = 200 #Capas del modelo

#Definimos el modelo
modelo = tf.keras.Sequential(([

                                #Cada observacion es de 28x28x1 (imagenes de 28x28 pixeles que corresponden a un numero del 0 al 9), por lo que es un tensor de rango 3
                                # Con el metodo Faltten achatamos el tensor de rango tres para convertirlo en un vector
                                #Esto nos permite crear una red neuronal de proalimentacion
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),

                                #tf.keras.layers.Dense esta basicamente impplementando la operacion: salida = activation(dot(entradat,peso) + sesgo)
                                tf.keras.layers.Dense(tamaño_capa_oculta,activation='relu'), #Primera capa oculta con la funcion de activacion ReLu
                                tf.keras.layers.Dense(tamaño_capa_oculta,activation='tanh'), #Segunda capa oculta con la funcion de activacion TanH
                                #tf.keras.layers.Dense(tamaño_capa_oculta,activation='sigmoid'), #Tercera capa oculta con la funcion de activacion sigmoide
                                tf.keras.layers.Dense(tamaño_capa_oculta,activation='relu'), #Cuarta capa oculta
                                tf.keras.layers.Dense(tamaño_capa_oculta,activation='relu'), #Quinta capa oculta

                                #Capa de salida en valores de probabilidades
                                tf.keras.layers.Dense(tamaño_salida,activation='softmax')

                                ]))

Como podemos ver en esta seccion donde definimos a nuestro modelo podemos agregarle tantas capas ocultas como queramos y definir las funciones de activacion que mejor se ajusten a nuestro modelo, en mi caso encontre que la funcion sigmoide no es la mas optima para este tipo de red nuronal.

## Optimizador y fucion de Perdida

In [102]:
 #Utilizaremos el optimizador adam "Estimacion adaptativa del momentum"
 #Utilizaremos la funcion de perdida "Entropia cruzada categorica dispersa" ya que aplica a los datos con codificacion one-hot
modelo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Entrenamiento del modelo


In [103]:
NUM_EPOCAS = 5
modelo.fit(datos_entrenamiento, epochs=NUM_EPOCAS, validation_data=(entradas_validacion,objetivos_validacion),verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.2472 - accuracy: 0.9248 - val_loss: 0.1071 - val_accuracy: 0.9675 - 8s/epoch - 16ms/step
Epoch 2/5
540/540 - 6s - loss: 0.1002 - accuracy: 0.9684 - val_loss: 0.0847 - val_accuracy: 0.9745 - 6s/epoch - 11ms/step
Epoch 3/5
540/540 - 6s - loss: 0.0707 - accuracy: 0.9781 - val_loss: 0.0635 - val_accuracy: 0.9798 - 6s/epoch - 11ms/step
Epoch 4/5
540/540 - 7s - loss: 0.0547 - accuracy: 0.9829 - val_loss: 0.0541 - val_accuracy: 0.9827 - 7s/epoch - 13ms/step
Epoch 5/5
540/540 - 6s - loss: 0.0478 - accuracy: 0.9850 - val_loss: 0.0535 - val_accuracy: 0.9838 - 6s/epoch - 10ms/step


Como es probable que hayamos sobreajustado el modelo ajustando los hiper parametros vamos a probar nuestro modelo con los datos de prueba

# Probando el modelo


In [104]:
# .evaluate() da los valores de perdida y metrica para el modelo en modo de prueba
perdida_prueba,presicion_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 2s 2s/step - loss: 0.0836 - accuracy: 0.9769


Obtuvimos una presicion del 97.7% con nuestro modelo implementado a un conjunto de datos que hasta ahora, el modelo no "conocia" por lo que podemos decir que nuestro modelo es bastante bueno.